Napisać makro, które wypełni tablicę elementami ciągu:
- ciąg jest zdefiniowany w postaci wyrażenia opisującego zależność rekurencyjną.
- tablica jest wcześniej zaalokowana, zawiera również początkowe wartości ciągu.

Przykładowe działanie gotowego makra. Poniższy kod:

y = zeros(10)

y[1:2] = [1, 1]

@fill_series y[t] = y[t-1] + y[t-2]

println(y)

powinien wypisać
[1.0,1.0,2.0,3.0,5.0,8.0,13.0,21.0,34.0,55.0]

In [1]:
macro fill_series_1(expr)
  left, right = expr.args
  i = left.args[2]
  quote
    for $i in 1:(length($(left.args[1])))
      try $expr
        catch BoundsError
      end
    end
  end
end

In [2]:
macro fill_series_2(expr)
  minval = index_range(string(expr))[1]
  maxval = index_range(string(expr))[2]
  quote
    t = $minval
    while t <= $maxval
      $expr
      t += 1
    end
  end
end

In [3]:
dump(parse("x[t] = x[t-1] * 2 + x[t-2]"))

Expr 
  head: Symbol =
  args: Array(Any,(2,))
    1: Expr 
      head: Symbol ref
      args: Array(Any,(2,))
        1: Symbol x
        2: Symbol t
      typ: Any
    2: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol +
        2: Expr 
          head: Symbol call
          args: Array(Any,(3,))
          typ: Any
        3: Expr 
          head: Symbol ref
          args: Array(Any,(2,))
          typ: Any
      typ: Any
  typ: Any


In [4]:
function index_range(expr::AbstractString) 
  min = 0
  max = 0
   
  s = parse(expr)
  symbol = s.args[1].args[1]
  
  for i in s.args[2].args
    if(typeof(i) == Expr)
      tmp = i.args[2]

      if(tmp.args[1] == symbol)
        tmp = tmp.args[2]
      end

      s = string(tmp)
      s = s[3:length(s)]
      val = parse(Int64,s,10)

      if(val < min)
        min = val
      elseif(val > max)
        max = val
      end

    end
  end

  min = abs(min) + 1
  max = N - max
  return [min, max]
end

index_range (generic function with 1 method)

In [5]:
const N = 10
y = zeros(N)
y[1:2] = [1,1]

2-element Array{Int64,1}:
 1
 1

In [6]:
@fill_series_1 y[t] = y[t-1] + y[t-2]
println("Evaluating y[t] = y[t-1] + y[t-2]")
println(y)

Evaluating y[t] = y[t-1] + y[t-2]
[1.0,1.0,2.0,3.0,5.0,8.0,13.0,21.0,34.0,55.0]


In [7]:
z = zeros(N)
z[1:2] = [1,1]

2-element Array{Int64,1}:
 1
 1

In [8]:
@fill_series_2 z[t] = z[t-1] + z[t-2]
println("\nEvaluating z[t] = z[t-1] + z[t-2]")
println(z)


Evaluating z[t] = z[t-1] + z[t-2]
[1.0,1.0,2.0,3.0,5.0,8.0,13.0,21.0,34.0,55.0]


In [9]:
x = zeros(N)
x[1:2] = [1,1]

2-element Array{Int64,1}:
 1
 1

In [10]:
@fill_series_1 x[t] = x[t-1] * 2 + x[t-2]
println("\nEvaluating x[t] = x[t-1] * 2 + x[t-2]")
println(x)


Evaluating x[t] = x[t-1] * 2 + x[t-2]
[1.0,1.0,3.0,7.0,17.0,41.0,99.0,239.0,577.0,1393.0]


In [11]:
c = zeros(N)
c[1:2] = [1,1]

2-element Array{Int64,1}:
 1
 1

In [12]:
@fill_series_2 c[t] = c[t-1] / 2 + c[t-2]
println("\nEvaluating x[t] = c[t-1] * 2 + c[t-2]")
println(c)


Evaluating x[t] = c[t-1] * 2 + c[t-2]
[1.0,1.0,1.5,1.75,2.375,2.9375,3.84375,4.859375,6.2734375,7.99609375]
